<a href="https://colab.research.google.com/github/rmit-ir/Tutotrial-Practical-LLMs/blob/main/LLM_Tutorial_Challenge2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Required Python Packages

This script installs three essential Python packages:

1. **openai** - Enables interaction with OpenAI's API for AI-powered applications.
2. **requests** - A simple HTTP library for sending API requests and handling responses.
3. **serpapi** - A client library for the SerpAPI service, which allows programmatic access to Google search results.

These packages are necessary for AI-based applications, making API calls, and retrieving structured search engine results.

## Installation Commands

Run the following commands in the Notebook to install the required packages.


In [ ]:
# Install the SerpAPI client to retrieve structured Google search results
%pip install serpapi

# Install the BeautifulSoup library for parsing HTML and XML documents
%pip install beautifulsoup4

%pip install selenium webdriver-manager

In [68]:
# Import the requests library for making HTTP requests to APIs
import requests

# Import the JSON module for handling JSON data
import json

# Import the SerpAPI client for Google search results
from serpapi import GoogleSearch

# Import pandas for data manipulation and analysis
import pandas as pd

from google.colab import (
    userdata,
)  # Provides access to user-specific information in Google Colab, used to access the user's secret API key

import textwrap  # Used for formatting and wrapping text, useful for displaying text in a readable way

# from IPython.core.display import display, HTML

# Import BeautifulSoup from bs4 for parsing HTML and XML documents
from bs4 import BeautifulSoup

ImportError: cannot import name 'GoogleSearch' from 'serpapi' (/opt/homebrew/Caskroom/miniconda/base/envs/dev/lib/python3.10/site-packages/serpapi/__init__.py)

In [2]:
VERBOSE = 0  # 'VERBOSE' controls the level of logging or output that is displayed (0: no output, 1: some output, 2: all output)

# set line wrap for print, lower for smaller screens
WRAP = 100  # Defines the maximum line width for wrapping text
printw = lambda x: print(
    textwrap.fill(x, WRAP)
)  # Create a lambda function that wraps text to fit within the specified width (WRAP)

# Fetching Documents from SerpAPI

This function retrieves search results from Google using SerpAPI and structures them into a Pandas DataFrame.

## **Functionality**
- Uses SerpAPI to fetch search results based on a query.
- Extracts document titles, links, and snippets.
- Returns the data in a structured Pandas DataFrame.

## **Parameters**
- `query` *(str)*: The search query string.
- `num_results` *(int, optional)*: Number of search results to fetch (default: 10).

## **Returns**
- `pd.DataFrame`: A DataFrame containing document titles, links, and snippets.

In [ ]:
# Test that the API keys are set and accessible
assert (
    userdata.get("OPENROUTER_API_KEY") is not None
), "Please set your OPENROUTER_API_KEY key in user secrets and allow access to it."


assert userdata.get("SERP_API_KEY"), "Please set your SERP_API_KEY in user secrets."

OPENROUTER_API_KEY = userdata.get("OPENROUTER_API_KEY")
SERP_API_KEY = userdata.get("SERP_API_KEY")

In [ ]:
def fetch_documents_from_serpapi(query, num_results=10):
    """
    Fetch documents from SerpApi based on the query.

    Args:
        query (str): The search query string.
        num_results (int): The number of search results to fetch (default is 10).

    Returns:
        pd.DataFrame: DataFrame containing the document titles and links.
    """

    # Define the search parameters, more info at https://serpapi.com/google-light-api#api-parameters
    params = {
    "engine": "google_light",
    "q": query,  # Search query
    "num": num_results,  # Max number of results to retrieve
    "google_domain": "google.com", # Google domain to use for the search
    "hl": "en", # Language code
    "gl": "us", # Country code
    "api_key": SERP_API_KEY # Your SerpApi API key
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results["organic_results"]

    # save to a DataFrame
    df = pd.DataFrame(organic_results)

    # Extract the relevant data (document titles and links)
    documents = []
    for result in results.get("organic_results", []):
        documents.append(
            {
                "title": result.get("title"),
                "link": result.get("link"),
                "snippet": result.get("snippet", ""),
            }
        )
    # Pretty print the results
    if VERBOSE > 0:
        printw(f"Search results for query '{query}':")
        for doc in documents:
            printw(f"Title: {doc['title']}")
            printw(f"Link: {doc['link']}")
            printw(f"Snippet: {doc['snippet']}")
            print("-" * 80)

    # Convert to DataFrame for better readability
    df = pd.DataFrame(documents)

    return df

In [ ]:
# Example usage: Search for "Python"
query = "Python"
# documents_df = fetch_documents_from_serpapi(query)
documents_df = pd.read_csv("Testing_docs_df.csv")

# Display the results
documents_df
# documents_df.to_csv('Testing_docs_df.csv')

,Unnamed: 0,title,link,snippet
0,0,Welcome to Python.org,https://www.python.org/,Python is a programming language that lets you...
1,1,Python Tutorial,https://www.w3schools.com/python/,Learn Python. Python is a popular programming ...
2,2,"Online Python - IDE, Editor, Compiler, Interpr...",https://www.online-python.com/,Build and Run your Python code instantly. Onli...
3,3,Python - Visual Studio Marketplace,https://marketplace.visualstudio.com/items?ite...,The Python extension provides pluggable access...
4,4,python/cpython: The Python programming language,https://github.com/python/cpython,"Documentation for Python 3.14 is online, updat..."
5,5,Python (programming language),https://en.wikipedia.org/wiki/Python_(programm...,"Python is a high-level, general-purpose progra..."


# Display results in Google-like way

In [6]:
def create_google_serp_like_page(documents_df):
    """
    Display a Google-like search results page from the fetched documents.

    Args:
        documents_df (pd.DataFrame): DataFrame containing the document titles, links, and snippets.
    """

    # Create an HTML structure for displaying the search results in a Google-like layout
    html_content = """
    <html>
    <head>
        <title>Google Search Results</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                margin: 20px;
                background-color: #f9f9f9;
            }
            .search-results {
                max-width: 800px;
                margin: auto;
                background-color: white;
                padding: 20px;
                box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1);
                border-radius: 8px;
            }
            .result-item {
                margin-bottom: 20px;
            }
            .result-title {
                font-size: 20px;
                color: #1a0dab;
                text-decoration: none;
            }
            .result-title:hover {
                text-decoration: underline;
            }
            .result-snippet {
                color: #4d5156;
                font-size: 14px;
                margin-top: 5px;
            }
            .result-link {
                color: #006621;
                font-size: 14px;
            }
            .result-link:hover {
                text-decoration: underline;
            }
            .search-bar {
                background-color: #f8f9fa;
                background-image: url('https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png');
                padding: 10px;
                margin-bottom: 20px;
                border-radius: 8px;
                box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.1);
            }
            .search-bar input {
                width: 100%;
                padding: 10px;
                font-size: 16px;
                border-radius: 4px;
                border: 1px solid #ddd;
            }
        </style>
    </head>
    <body>
        <div class="search-results">
            <div class="search-bar">
                <input type="text" placeholder="Search Google..." value="Python" readonly>
            </div>
    """

    # Loop through each document and create a search result item
    for index, row in documents_df.iterrows():
        title = row["title"]
        link = row["link"]
        snippet = row["snippet"]

        html_content += f"""
            <div class="result-item">
                <a class="result-title" href="{link}" target="_blank">{title}</a>
                <div class="result-snippet">{snippet}</div>
                <a class="result-link" href="{link}" target="_blank">{link}</a>
            </div>
        """

    # Close the HTML tags
    html_content += """
        </div>
    </body>
    </html>
    """

    # Display the HTML content in the notebook
    # display(HTML(html_content))


# Example usage: Display the results from the fetched documents
create_google_serp_like_page(documents_df)

In [65]:
# Core Selenium package for browser automation
from selenium import webdriver
# Chrome-specific options for configuring the browser session
from selenium.webdriver.chrome.options import Options
# Locator strategies for finding elements on the page
from selenium.webdriver.common.by import By
# Import WebDriverWait and expected conditions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Import time module for sleep functionality
import time

def fetch_and_parse_webpage(url, max_retries=2, timeout=30):
    """
    Fetch and parse webpage with improved timeout handling and retries.
    
    Args:
        url (str): The webpage URL to fetch
        max_retries (int): Number of retry attempts
        timeout (int): Page load timeout in seconds
    """
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    # Add performance options
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.page_load_strategy = 'eager'  # Wait for the page to load completely
    
    for attempt in range(max_retries):
        try:
            driver = webdriver.Chrome(options=chrome_options)
            driver.set_page_load_timeout(timeout)
            driver.set_script_timeout(timeout)
            
            # Load page with explicit wait
            driver.get(url)
            WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
            # List of CSS selectors for elements to remove (navigation, menus, etc.)
            ignore_elements = [
                "nav",  # Navigation bars
                "header",  # Site headers
                "footer",  # Site footers
                "menu",  # Menu elements
                '[role="navigation"]',  # ARIA navigation roles
                '[role="banner"]',  # ARIA banner roles (headers)
                '[role="complementary"]',  # ARIA sidebars/complementary content
                ".sidebar",  # Sidebar classes
                "#navigation",  # Navigation IDs
                ".menu",  # Menu classes
                ".nav",  # Nav classes
            ]

            # Remove all non-content elements from the page
            for selector in ignore_elements:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    try:
                        driver.execute_script("arguments[0].remove()", element)
                    except:
                        continue

            # Prioritized list of selectors for main content areas
            content_selectors = [
                "article",  # Standard article tag
                '[role="main"]',  # ARIA main content role
                ".post-content",  # Common content class
                ".article-content",  # Common article class
                "main",  # HTML5 main tag
                "#content",  # Common content ID
            ]

            # Try each content selector in order until we find content
            content = None
            for selector in content_selectors:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                if elements:
                    content = elements[0].text
                    break

            # If no content found with specific selectors, get all body text
            if not content:
                content = driver.find_element(By.TAG_NAME, "body").text

            driver.quit()

            # Clean and format the extracted text
            lines = [line.strip() for line in content.split("\n")]
            # Remove short lines (likely UI elements) and empty lines
            lines = [
                line for line in lines if line and len(line) > 20
            ]
            return "\n".join(lines)
        
        except requests.exceptions.Timeout:
            print(f"Timeout occurred for the URL: {url}")
            driver.quit()
            if attempt == max_retries - 1:
                return "Failed to fetch the webpage due to timeout."
            chrome_options.page_load_strategy = 'eager'  # Reset to eager strategy
            time.sleep(3)  # Wait before retrying
            print(f"Retrying... ({attempt + 1}/{max_retries})")
            continue
        except Exception as e:
            print(f"Attempt {attempt + 1}/{max_retries} failed for the URL: {url}")
            print("Error:", str(e))
            driver.quit()
            if attempt == max_retries - 1:
                return "Failed to fetch the webpage."
            time.sleep(3)  # Wait before retrying
            print(f"Retrying... ({attempt + 1}/{max_retries})")
            continue
            
    return "Failed to fetch the webpage."


# Example usage
url = "https://www.allsides.com/story/john-mccain-end-treatment-brain-cancer"
webpage_text = fetch_and_parse_webpage(url)

if webpage_text:
    print("Extracted content:")
    print("-" * 80)
    printw(webpage_text[:1000])  # Show first 1000 characters
    print("-" * 80)
else:
    print("Failed to fetch the webpage.")

Extracted content:
--------------------------------------------------------------------------------
Verifying you are human. This may take a few seconds. www.allsides.com needs to review the security
of your connection before proceeding.
--------------------------------------------------------------------------------


In [66]:
documents_df = documents_df.assign(text=documents_df["link"].apply(fetch_and_parse_webpage))
documents_df

,Unnamed: 0,title,link,snippet,text
0,0,Welcome to Python.org,https://www.python.org/,Python is a programming language that lets you...,"Join us in Pittsburgh, PA starting May 14, 202..."
1,1,Python Tutorial,https://www.w3schools.com/python/,Learn Python. Python is a popular programming ...,HTML CSS JAVASCRIPT SQL PYTHON JAVA PHP HOW TO...
2,2,"Online Python - IDE, Editor, Compiler, Interpr...",https://www.online-python.com/,Build and Run your Python code instantly. Onli...,🚀 Upgrade your coding experience! Try our new ...
3,3,Python - Visual Studio Marketplace,https://marketplace.visualstudio.com/items?ite...,The Python extension provides pluggable access...,Visual Studio Code>Programming Languages>Pytho...
4,4,python/cpython: The Python programming language,https://github.com/python/cpython,"Documentation for Python 3.14 is online, updat...",This is Python version 3.14.0 alpha 6\nCopyrig...
5,5,Python (programming language),https://en.wikipedia.org/wiki/Python_(programm...,"Python is a high-level, general-purpose progra...","From Wikipedia, the free encyclopedia\nParadig..."


In [67]:
for text in documents_df["text"]:
    printw(text[:1000])  # Show first 1000 characters
    print("-" * 80)

Join us in Pittsburgh, PA starting May 14, 2025. Grab your ticket today before we sell out!
REGISTER FOR PYCON US! Whether you're new to programming or an experienced developer, it's easy to
learn and use Python. Start with our Beginner’s Guide Python source code and installers are
available for download for all versions! Latest: Python 3.13.2 Documentation for Python's standard
library, along with tutorials and guides, are available online. Looking for work or have a Python
related position that you're trying to hire for? Our relaunched community-run job board is the place
to go. Thanks to the flexibility of Python and the powerful ecosystem of packages, the Azure CLI
supports features such as autocompletion (in shells that support it), persistent credentials,
JMESPath result parsing, lazy initialization, network-less unit tests, and more. Building an open-
source and cross-platform Azure CLI with Python by Dan Taylor >>> Python Software Foundation The
mission of the Python Software
-

In [ ]:
def get_response(
    prompt: str, model: str, verbose: int = VERBOSE, **model_kwargs
) -> dict:
    """
    Get a response from the OpenRouter API using the given prompt and model.
    Make sure to set your OpenRouter API key in the environment variable
    OPENROUTER_API_KEY. OpenRouter normalizes requests and responses across
    providers. That is, you can use the same code to call different models from
    different providers.
    Args:
        prompt (str): The prompt to send to the model.
        model (str): The model to use.
        verbose (int): Verbosity level for debugging.
        **model_kwargs: Additional keyword arguments for the model.
            - top_p: Top-p sampling parameter.
            - temperature: Temperature parameter for sampling.
            - frequency_penalty: Frequency penalty parameter.
            - presence_penalty: Presence penalty parameter.
            - repetition_penalty: Repetition penalty parameter.
            - top_k: Top-k sampling parameter.
            - max_tokens: Maximum number of tokens to generate.
    Note: The model_kwargs parameters are optional and will be set to default values if not provided.
    Returns:
        dict: The response from the model.
    """
    # Check if model parameter is provided, if not, set a default value.
    # More information about the parameters can be found in the OpenRouter API documentation.
    # https://openrouter.ai/docs/api-reference/parameters
    top_p = model_kwargs.get("top_p", 1)
    temperature = model_kwargs.get("temperature", 0.9)
    frequency_penalty = model_kwargs.get("frequency_penalty", 0)
    presence_penalty = model_kwargs.get("presence_penalty", 0)
    repetition_penalty = model_kwargs.get("repetition_penalty", 1)
    top_k = model_kwargs.get("top_k", 0)
    max_tokens = model_kwargs.get("max_tokens", 1000)

    messages = [{"role": "user", "content": prompt}]

    response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={"Authorization": f"Bearer {userdata.get('OPENROUTER_API_KEY')}"},
        data=json.dumps(
            {
                "model": model,
                "messages": messages,
                "top_p": top_p,
                "temperature": temperature,
                "frequency_penalty": frequency_penalty,
                "presence_penalty": presence_penalty,
                "repetition_penalty": repetition_penalty,
                "top_k": top_k,
                "max_tokens": max_tokens,
            }
        ),
    )
    if verbose > 0:
        print(f"Response status code: {response.status_code}")
    response_json = response.json()
    # let's print how many tokens we used, it can be useful for cost estimation
    if verbose > 0:
        print(f"Response usage: {response_json.get('usage')}")
    return response_json

In [ ]:
def generate_documents_summary(
    documents_df: pd.DataFrame,
    prompt: str,
    model: str,
    verbose: int = VERBOSE,
) -> tuple:
    """
    Generate relevance predictions using the OpenRouter API.
    Args:
        qrel_df (DataFrame): The qrel DataFrame.
        docs (DataFrame or Lz4FullStore): The document store.
        query_df (DataFrame): The query DataFrame.
        prompt (str): The prompt template.
        model (str): The model to use.
        verbose (int): Verbosity level for debugging.
    Returns:
        dict: The relevance predictions.
        dict: The reasoning results.
        list: The raw responses from the model.
    """
    results = {}
    reasoning_results = {}
    raw_responses = []  # for debugging, logging and backup

    for index, row in documents_df.iterrows():
        qid, doc_id = row["query_id"], row["doc_id"]
        if isinstance(docs, ir_datasets.indices.lz4_pickle.PickleLz4FullStore):
            doc = docs.get(doc_id)
        else:
            doc = docs.loc[doc_id]
        prompt_text = prompt.format(
            query=query_df.loc[qid, "text"], document=doc.text
        )  # adding a query and document to the prompt
        if verbose > 0:
            print(f"Running for {qid, doc_id}")
            if verbose > 1:
                print(f"Prompt for {qid, doc_id}:")
                printw(prompt_text)
        response_json = get_response(prompt=prompt_text, model=model, verbose=verbose)
        raw_responses.append(response_json)
        response_message = response_json.get("choices", {0: {"message": None}})[0][
            "message"
        ]
        if response_message is None:
            print(f"No response message for {qid, doc_id}")
            print(response_json.get("error"))
            continue
        if verbose > 1:
            print(f"Response for {qid, doc_id}:")
            printw(f"Response message: {response_message}")
            print("\n", "-=" * 5, " End of response ", "=-" * 5, "\n")
        response_content = response_message.get("content", None)
        reasoning_results[qid, doc_id] = response_message.get("reasoning", None)
        results[qid, doc_id] = response_content
    return results, reasoning_results, raw_responses

In [ ]:
def summarize_document(document_text):
    """
    Summarizes the document using the OpenRouter API.

    Args:
        document_text (str): The text of the document to be summarized.
        OPENROUTER_API_KEY: The API key for OpenRouter authentication.

    Returns:
        str: The summary of the document, or None if an error occurs.
    """

    # Define the API endpoint for OpenRouter
    openrouter_endpoint = "https://openrouter.ai/api/v1/chat/completions"

    # Define request headers with API authorization
    headers = {
        "Authorization": f"Bearer {userdata.get('OPENROUTER_API_KEY')}",
        "Content-Type": "application/json",
    }

    # Define the request payload
    payload = {
        "model": "gpt-4o-mini",  # Model selection
        "messages": [
            {
                "role": "system",
                "content": "You are an AI assistant that summarizes documents.",
            },
            {
                "role": "user",
                "content": f"Summarize the following text:\n\n{document_text}",
            },
        ],
        "max_tokens": 200,  # Limit response length
        "temperature": 0.7,  # Control randomness
    }

    try:
        # Send the request to OpenRouter API
        response = requests.post(openrouter_endpoint, headers=headers, json=payload)

        # Parse the response
        if response.status_code == 200:
            summary = (
                response.json()
                .get("choices", [{}])[0]
                .get("message", {})
                .get("content", "")
                .strip()
            )
            return summary
        else:
            print(
                f"Error summarizing document: {response.status_code} - {response.text}"
            )
            return None

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

In [ ]:
# Loop through each document and summarize it
documents_df["summary"] = documents_df["snippet"].apply(lambda x: summarize_document(x))

# Display the documents with their summaries
documents_df[["title", "link", "summary"]].head()